In [70]:
import sys
import json
import os
import numpy as np
os.environ['PYSPARK_PYTHON']="/home/work/local/anaconda3/bin/python3.6"
import pandas as pd
from pyspark import SparkContext, SparkConf
from pyspark.sql import HiveContext, DataFrame
from datetime import date, timedelta

from pyspark.sql import functions as f
from pyspark import SparkContext, SparkConf
from pyspark.sql import HiveContext,SparkSession
from datetime import datetime,timedelta,date
import matplotlib.pyplot as plt
pd.set_option('display.max_columns',None)

In [71]:
def get_host_and_ip():
    try:
        import socket

        hostname = socket.gethostname()  # 获取当前主机名
        # 通过hostname查询,注意这个并不一定会得到真确的IP地址
        ip = socket.gethostbyname(socket.gethostname())
        return hostname, ip
    except:
        return "unknown_host", "0.0.0.0"
hostname, ip=get_host_and_ip()

builder = SparkSession.builder
# 队列要切换成事业部自己的队列
# builder.config("spark.yarn.queue",'root.wangxiao.analysis')
builder.config("spark.yarn.queue",'root.wangxiao.chanpin')
# builder.config("spark.yarn.queue",'root.default')

builder.config("spark.driver.host",ip)
# 根据任务调整自己的资源到合理范围
builder.config("spark.executor.memory", '4g')
builder.config("spark.driver.memory", '4g')
builder.config("spark.executor.instances", 40)
builder.config("spark.executor.cores", 2)
# builder.master(master_url)
spark = builder.enableHiveSupport().appName("demo").getOrCreate()


In [72]:
start='2022-07-25'
end='2022-07-31'
# start_1=datetime.now() - timedelta(days=7)
# start = datetime.strftime(start_1,'%Y-%m-%d')
# end_1=datetime.now() - timedelta(days=1)
# end = datetime.strftime(end_1,'%Y-%m-%d')

# 开始时间
start_n=(pd.to_datetime(datetime.strftime(datetime.now(),'%Y-%m-%d'))-pd.to_datetime(start)).days
start_time=datetime.now() - timedelta(days=start_n)
start_date = datetime.strftime(start_time,'%m.%d')
start_yes = datetime.strftime(start_time,'%Y%m%d')
start_yes_fmt = datetime.strftime(start_time,'%Y-%m-%d')

# 结束时间
end_n=(pd.to_datetime(datetime.strftime(datetime.now(),'%Y-%m-%d'))-pd.to_datetime(end)).days
end_time=datetime.now() - timedelta(days=end_n)
end_date = datetime.strftime(end_time,'%m.%d')
end_yes = datetime.strftime(end_time,'%Y%m%d')
end_yes_fmt = datetime.strftime(end_time,'%Y-%m-%d')

# 一个月前
end_29=datetime.now() - timedelta(days=end_n+29)
end_yes_29 = datetime.strftime(end_29,'%Y%m%d')
end_yes_29_fmt = datetime.strftime(end_29,'%Y-%m-%d')

# 半个月前
end_14=datetime.now() - timedelta(days=start_n+14)
end_yes_14 = datetime.strftime(end_14,'%Y%m%d')
end_yes_14_fmt = datetime.strftime(end_14,'%Y-%m-%d')

# 7天前
end_7=datetime.now() - timedelta(days=start_n+8)
end_yes_7 = datetime.strftime(end_7,'%Y%m%d')
end_yes_7_fmt = datetime.strftime(end_7,'%Y-%m-%d')

# 开始1天前
end_1=datetime.now() - timedelta(days=start_n+1)
end_yes_1 = datetime.strftime(end_1,'%Y%m%d')
end_yes_1_fmt = datetime.strftime(end_1,'%Y-%m-%d')

end_yes_fmt


'2022-07-31'

### 日活

In [73]:
# 首页日活
sql = """
select
    sum(a.`累计日活`) as `累计日活`
from
(
    select 
        dt,
        count(distinct active_user) as `累计日活`

    from wx_dm_dp.dm_visit_label_dau
    where 
        -- dt>=date_format(date_add(current_date,-45),'yyyyMMdd')
        -- and dt<=date_format(date_add(current_date,-39),'yyyyMMdd')
        dt>='%s' and dt<='%s'
        and big_client='APP'
    group by dt
) a

""" % (start_yes,end_yes)

result = spark.sql(sql)

data_rihuo=result.toPandas()
data_rihuo.head()


,累计日活
0,1099679


### 曝光

In [74]:
# 创新资源位曝光UV
sql = """
select 
    case when adver_id in ('105855','105856','105857','105917','105925','105957','106005','106023','106142','106000','106004','106143','106159','106213','106199','106198') 
    then '自然博物' else line_name end as `业务分类`
    ,fk_source
    ,sum(show_uv) as `曝光UV`
from wx_adsdb.ads_mkt_bdp_app_mkt_adserving_summary_bdp_new_di_id a
where 
    flag IN ('6')
    AND (client_type_new IN ('app','') or client_type_new is null)
    -- AND dt >= date_format(date_add(current_date,-45),'yyyyMMdd')
    -- AND dt <= date_format(date_add(current_date,-39),'yyyyMMdd')
    and dt>='%s' and dt<='%s'
    and (line_name in ('ART','TED','航天','自然博物') or adver_id in ('105855','105856','105857','105917','105925','105957','106005','106023','106142','106000','106004','106143','106159','106213','106199','106198'))
    and page_name='首页'
    and adslot_id in ('99','98','97','96','95','94','93','92','91','90','81','80','79','78','77','76','68','46','64','57','58','56','62','59','60')
group by
    case when adver_id in ('105855','105856','105857','105917','105925','105957','106005','106023','106142','106000','106004','106143','106159','106213','106199','106198') 
    then '自然博物' else line_name end,
    fk_source

""" % (start_yes,end_yes)

result = spark.sql(sql)

data_baoguang_chuangxin=result.toPandas()
data_baoguang_chuangxin.head()


,业务分类,fk_source,曝光UV
0,自然博物,635378319,18
1,ART,635457000,8447
2,ART,635456919,141173
3,TED,635462711,38209
4,自然博物,635342170,5


In [75]:
# 素质导流课资源位曝光UV
sql = """
select 
    specific_name as `业务分类all`
    ,fk_source
    ,sum(show_uv) as `曝光UV`
from wx_adsdb.ads_mkt_bdp_app_mkt_adserving_summary_bdp_new_di_id a
where 
    flag IN ('6')
    AND (client_type_new IN ('app','') or client_type_new is null)
    -- AND dt >= date_format(date_add(current_date,-45),'yyyyMMdd')
    -- AND dt <= date_format(date_add(current_date,-39),'yyyyMMdd')
    and dt>='%s' and dt<='%s'
    and adver_id not in ('105855','105856','105857','105917','105925','105957','106005','106023','106142','106000','106004','106143','106159','106213','106199','106198')
    and line_name in ('素质爆品课','素质特训班')
    and page_name='首页'
    and adslot_id in ('99','98','97','96','95','94','93','92','91','90','81','80','79','78','77','76','68','46','64','57','58','56','62','59','60')
group by
    specific_name,
    fk_source
    
""" % (start_yes,end_yes)

result = spark.sql(sql)

data_baoguang_suzhi=result.toPandas()
data_baoguang_suzhi['业务分类']=''
for i in np.arange(len(data_baoguang_suzhi)):
    data_baoguang_suzhi.loc[i,'业务分类']=data_baoguang_suzhi.iloc[i]['业务分类all'].split("-")[1]
data_baoguang_suzhi=data_baoguang_suzhi[data_baoguang_suzhi['业务分类'].isin(['人文美育','科学创想','编程','演讲与口才','国际象棋'])]
data_baoguang_suzhi.head()


,业务分类all,fk_source,曝光UV,业务分类
0,爆品课-科学创想,635438748,62309,科学创想
1,爆品课-编程-0元,635054395,23,编程
2,特训班-编程,629740456,1,编程
4,特训班-编程,629742194,3,编程
5,爆品课-演讲与口才,635402172,963,演讲与口才


In [76]:
# 老带新资源位曝光UV
sql = """
select 
    line_name as `业务分类`
    ,fk_source
    ,sum(show_uv) as `曝光UV`
from wx_adsdb.ads_mkt_bdp_app_mkt_adserving_summary_bdp_new_di_id a
where 
    flag IN ('6')
    AND (client_type_new IN ('app','') or client_type_new is null)
    -- AND dt >= date_format(date_add(current_date,-45),'yyyyMMdd')
    -- AND dt <= date_format(date_add(current_date,-39),'yyyyMMdd')
    and dt>='%s' and dt<='%s'
    and line_name in ('常规老带新','特训班老带新','专题老带新')
    and adver_id not in ('105855','105856','105857','105917','105925','105957','106005','106023','106142','106000','106004','106143','106159','106213','106199','106198')
    and page_name='首页'
    and adslot_id in ('99','98','97','96','95','94','93','92','91','90','81','80','79','78','77','76','68','46','64','57','58','56','62','59','60')
group by
    line_name,
    fk_source
    
""" % (start_yes,end_yes)

result = spark.sql(sql)

data_baoguang_laodaixin=result.toPandas()
# data_baoguang_laodaixin['业务分类']=''
# for i in np.arange(len(data_baoguang_laodaixin)):
#     data_baoguang_laodaixin.loc[i,'业务分类']=data_baoguang_laodaixin.iloc[i]['业务分类all'].split("-")[1]
data_baoguang_laodaixin.head()


,业务分类,fk_source,曝光UV
0,特训班老带新,634967202,107
1,特训班老带新,628571788,2
2,常规老带新,626336218,1
3,特训班老带新,623239739,2
4,特训班老带新,628187070,9


In [77]:
# BOX资源位曝光UV
sql = """
select 
    'BOX' as `业务分类`
    ,fk_source
    ,sum(show_uv) as `曝光UV`
from wx_adsdb.ads_mkt_bdp_app_mkt_adserving_summary_bdp_new_di_id a
where 
    flag IN ('6')
    AND (client_type_new IN ('app','') or client_type_new is null)
    -- AND dt >= date_format(date_add(current_date,-45),'yyyyMMdd')
    -- AND dt <= date_format(date_add(current_date,-39),'yyyyMMdd')
    and dt>='%s' and dt<='%s'
    and adver_id not in ('105855','105856','105857','105917','105925','105957','106005','106023','106142','106000','106004','106143','106159','106213','106199','106198')
    and line_name in ('大师课','素养课')
    and page_name='首页'
    and adslot_id in ('99','98','97','96','95','94','93','92','91','90','81','80','79','78','77','76','68','46','64','57','58','56','62','59','60')
group by
    'BOX',
    fk_source

""" % (start_yes,end_yes)

result = spark.sql(sql)

data_baoguan_box=result.toPandas()
data_baoguan_box.head()


,业务分类,fk_source,曝光UV
0,BOX,635442454,118810
1,BOX,635380252,3
2,BOX,628195667,1
3,BOX,635364198,1
4,BOX,628451923,2


In [78]:
# 全真资源位曝光UV
sql = """
select 
    '全真' as `业务分类`
    ,fk_source
    ,sum(show_uv) as `曝光UV`
from wx_adsdb.ads_mkt_bdp_app_mkt_adserving_summary_bdp_new_di_id a
where 
    flag IN ('6')
    AND (client_type_new IN ('app','') or client_type_new is null)
    -- AND dt >= date_format(date_add(current_date,-45),'yyyyMMdd')
    -- AND dt <= date_format(date_add(current_date,-39),'yyyyMMdd')
    and dt>='%s' and dt<='%s'
    and adver_id not in ('105855','105856','105857','105917','105925','105957','106005','106023','106142','106000','106004','106143','106159','106213','106199','106198')
    and (line_name in ('口才','素材','next-科学','情商','增长','外部大V') or specific_name in ('公益活动'))
    and page_name='首页'
    and adslot_id in ('99','98','97','96','95','94','93','92','91','90','81','80','79','78','77','76','68','46','64','57','58','56','62','59','60')
group by
    '全真',
    fk_source

""" % (start_yes,end_yes)

result = spark.sql(sql)

data_baoguang_next=result.toPandas()
data_baoguang_next.head()


,业务分类,fk_source,曝光UV
0,全真,635371948,8
1,全真,635456591,6
2,全真,635381452,2
3,全真,635456642,52
4,全真,635381462,8


In [79]:
# 图书资源位曝光UV
sql = """
select 
    '图书' as `业务分类`
    ,fk_source
    ,sum(show_uv) as `曝光UV`
from wx_adsdb.ads_mkt_bdp_app_mkt_adserving_summary_bdp_new_di_id a
where 
    flag IN ('6')
    AND (client_type_new IN ('app','') or client_type_new is null)
    -- AND dt >= date_format(date_add(current_date,-45),'yyyyMMdd')
    -- AND dt <= date_format(date_add(current_date,-39),'yyyyMMdd')
    and dt>='%s' and dt<='%s'
    and line_name in ('书城')
    and adver_id not in ('105855','105856','105857','105917','105925','105957','106005','106023','106142','106000','106004','106143','106159','106213','106199','106198')
    and page_name='首页'
    and adslot_id in ('99','98','97','96','95','94','93','92','91','90','81','80','79','78','77','76','68','46','64','57','58','56','62','59','60')
group by
    '图书',
    fk_source

""" % (start_yes,end_yes)

result = spark.sql(sql)

data_baoguang_tushu=result.toPandas()
data_baoguang_tushu.head()


,业务分类,fk_source,曝光UV
0,图书,635365817,14
1,图书,635392035,43283
2,图书,635344590,17
3,图书,635403811,43175
4,图书,635352662,10


### leads

In [80]:
# 素质导流课leads
sql = """
select
    c.subject_names as `业务分类`
    ,count(distinct c.stu_course_id) as `购买人次`
    -- ,coalesce(sum(a.pay_amt),0) as `GMV(截面)`
from
(
   select 
        '素质导流课' as `业务分类`,
        fk_source
    from wx_adsdb.ads_mkt_bdp_app_mkt_adserving_summary_bdp_new_di_id  t1
    where 
        flag IN ('6')
        AND (client_type_new IN ('app','') or client_type_new is null)
        -- AND dt >= date_format(date_add(current_date,-45),'yyyyMMdd')
        -- AND dt <= date_format(date_add(current_date,-39),'yyyyMMdd')
        and dt>='%s' and dt<='%s'
        and adver_id not in ('105855','105856','105857','105917','105925','105957','106005','106023','106142','106000','106004','106143','106159','106213','106199','106198')
        and line_name in ('素质爆品课','素质特训班')
        -- and page_name='首页'
        and adslot_id in ('99','98','97','96','95','94','93','92','91','90','81','80','79','78','77','76','68','46','64','57','58','56','62','59','60')
    group by
        '素质导流课',
        fk_source
) a

inner join
(
    SELECT 
        replace(substr(course_create_time,1,10),'-','') as `dt`
        ,stu_course_id
        ,stu_id
        ,hotspot_id
        ,subject_names
    FROM wx_dwddb.dwd_mkt_order_div_course_fd
    WHERE 
        dt=date_format(date_add(current_date,-1),'yyyy-MM-dd')
        AND subject_names IN ('人文美育','科学创想','编程','演讲与口才','国际象棋')
        AND order_return_time = '' --未退费
        and order_first_channel_name='站内'
        -- and substr(course_create_time,1,10) >=date_format(date_add(current_date,-44),'yyyy-MM-dd')
        -- and substr(course_create_time,1,10) <=date_format(date_add(current_date,-38),'yyyy-MM-dd')
        and substr(course_create_time,1,10) >= '%s' and substr(course_create_time,1,10) <= '%s'
) c on c.hotspot_id=a.fk_source
group by 
    c.subject_names


""" % (start_yes,end_yes,start_yes_fmt,end_yes_fmt)

result = spark.sql(sql)

data_leads_suzhi=result.toPandas()
data_leads_suzhi.head()


,业务分类,购买人次
0,演讲与口才,548
1,编程,2192
2,科学创想,1015
3,国际象棋,1452
4,人文美育,624


In [81]:
# 老带新leads
sql = """
select
    a.line_name as `业务分类`
    ,count(distinct c.stu_course_id) as `购买人次`
    -- ,coalesce(sum(a.pay_amt),0) as `GMV(截面)`
from
(
   select 
        line_name,
        fk_source
    from wx_adsdb.ads_mkt_bdp_app_mkt_adserving_summary_bdp_new_di_id  t1
    where 
        flag IN ('6')
        AND (client_type_new IN ('app','') or client_type_new is null)
        -- AND dt >= date_format(date_add(current_date,-45),'yyyyMMdd')
        -- AND dt <= date_format(date_add(current_date,-39),'yyyyMMdd')
        and dt>='%s' and dt<='%s'
        and adver_id not in ('105855','105856','105857','105917','105925','105957','106005','106023','106142','106000','106004','106143','106159','106213','106199','106198')
        and line_name in ('常规老带新','特训班老带新','专题老带新')
        and page_name='首页'
        and adslot_id in ('99','98','97','96','95','94','93','92','91','90','81','80','79','78','77','76','68','46','64','57','58','56','62','59','60')
    group by
        line_name,
        fk_source
) a

inner join
(
    SELECT 
        replace(substr(course_create_time,1,10),'-','') as `dt`
        ,stu_course_id
        ,stu_id
        ,hotspot_id
        ,subject_names
    FROM wx_dwddb.dwd_mkt_order_div_course_fd
    WHERE 
        dt=date_format(date_add(current_date,-1),'yyyy-MM-dd')
        AND subject_names IN ('人文美育','科学创想','编程','演讲与口才','国际象棋')
        AND order_return_time = '' --未退费
        and order_first_channel_name='老带新'
        -- and substr(course_create_time,1,10) >=date_format(date_add(current_date,-44),'yyyy-MM-dd')
        -- and substr(course_create_time,1,10) <=date_format(date_add(current_date,-38),'yyyy-MM-dd')
        and substr(course_create_time,1,10) >= '%s' and substr(course_create_time,1,10) <= '%s'
) c on c.hotspot_id=a.fk_source
group by 
    a.line_name

""" % (start_yes,end_yes,start_yes_fmt,end_yes_fmt)

result = spark.sql(sql)

data_leads_laodaixin=result.toPandas()
data_leads_laodaixin.head()


,业务分类,购买人次
0,常规老带新,1129
1,特训班老带新,282


In [82]:
# 创新leads
sql = """
select
    case when b.fk_tea_teacher in ('89254','89340','89182') then '自然博物' 
             when b.fk_tea_teacher in ('89309') then '航天'
             when b.fk_tea_teacher in ('89290') then 'ART' 
             when b.fk_tea_teacher in ('89334','89333','89307','89404') then 'TED'
             else '其他' end as `业务分类`
    ,count(distinct b.stu_course_id) as `购买人次`
from
(
    select 
        fk_source
    from wx_adsdb.ads_mkt_bdp_app_mkt_adserving_summary_bdp_new_di_id a
    where 
        flag IN ('6')
        AND (client_type_new IN ('app','') or client_type_new is null)
        -- AND dt >= date_format(date_add(current_date,-45),'yyyyMMdd')
        -- AND dt <= date_format(date_add(current_date,-39),'yyyyMMdd')
        and dt>='%s' and dt<='%s'
        and (line_name in ('ART','TED','航天','自然博物') or adver_id in ('105855','105856','105857','105917','105925','105957','106005','106023','106142','106000','106004','106143','106159','106213','106199','106198'))
        and page_name='首页'
        and adslot_id in ('99','98','97','96','95','94','93','92','91','90','81','80','79','78','77','76','68','46','64','57','58','56','62','59','60')
    group by
        fk_source
) a
left join
(
    select
        b.hotspot_id
        ,a.stu_course_id
        ,a.fk_tea_teacher
    from
    (
        select 
            distinct
            fk_tea_teacher
            ,stu_course_id
        from wx_dwsdb.dws_trd_sales_stucou_tag_all
        where 
            dt=date_format(date_add(current_date,-1),'yyyyMMdd')
            and is_return='0'
            -- and substr(course_create_time,1,10) >=date_format(date_add(current_date,-44),'yyyy-MM-dd')
            -- and substr(course_create_time,1,10) <=date_format(date_add(current_date,-38),'yyyy-MM-dd')
            and substr(course_create_time,1,10) >= '%s' and substr(course_create_time,1,10) <= '%s'
            and cast(course_amt as double)<50
            and fk_tea_teacher in ('89254','89340','89182','89309','89290','89334','89333','89307','89404')
    ) a
    left join
    (
        select 
            stu_course_id 
            ,hotspot_id
        from wx_dwddb.dwd_mkt_order_course_channel_fd
        where dt=date_format(date_add(current_date,-1),'yyyy-MM-dd')
    ) b on b.stu_course_id = a.stu_course_id
    group by 
        b.hotspot_id
        ,a.stu_course_id
        ,a.fk_tea_teacher
) b on a.fk_source=b.hotspot_id
group by 
    case when b.fk_tea_teacher in ('89254','89340','89182') then '自然博物' 
             when b.fk_tea_teacher in ('89309') then '航天'
             when b.fk_tea_teacher in ('89290') then 'ART' 
             when b.fk_tea_teacher in ('89334','89333','89307','89404') then 'TED'
             else '其他' end

""" % (start_yes,end_yes,start_yes_fmt,end_yes_fmt)

result = spark.sql(sql)

data_leads_chuangxin=result.toPandas()
data_leads_chuangxin.head()


,业务分类,购买人次
0,TED,118
1,ART,71
2,自然博物,1001
3,其他,0


In [83]:
# # 创新leads
# sql = """
# select
#     case when b.fk_tea_teacher in ('89254','89340') then '自然博物' 
#              when b.fk_tea_teacher in ('89309') then '航天'
#              when b.fk_tea_teacher in ('89290') then 'ART' 
#              when b.fk_tea_teacher in ('89334','89333','89307','89404') then 'TED'
#              else '其他' end as `业务分类`
#     ,b.stu_course_id
#     ,a.fk_source
# from
# (
#     select 
#         fk_source
#     from wx_adsdb.ads_mkt_bdp_app_mkt_adserving_summary_bdp_new_di_id a
#     where 
#         flag IN ('6')
#         AND (client_type_new IN ('app','') or client_type_new is null)
#         -- AND dt >= date_format(date_add(current_date,-45),'yyyyMMdd')
#         -- AND dt <= date_format(date_add(current_date,-39),'yyyyMMdd')
#         and dt>='%s' and dt<='%s'
#         and (line_name in ('ART','TED','航天','自然博物') or adver_id in ('105855','105856','105857','105917','105925','105957','106005','106023','106142','106000','106004','106143','106159','106213','106199','106198'))
#         and page_name='首页'
#     group by
#         fk_source
# ) a
# left join
# (
#     select
#         b.hotspot_id
#         ,a.stu_course_id
#         ,a.fk_tea_teacher
#     from
#     (
#         select 
#             distinct
#             fk_tea_teacher
#             ,stu_course_id
#         from wx_dwsdb.dws_trd_sales_stucou_tag_all
#         where 
#             dt=date_format(date_add(current_date,-1),'yyyyMMdd')
#             and is_return='0'
#             -- and substr(course_create_time,1,10) >=date_format(date_add(current_date,-44),'yyyy-MM-dd')
#             -- and substr(course_create_time,1,10) <=date_format(date_add(current_date,-38),'yyyy-MM-dd')
#             and substr(course_create_time,1,10) >= '%s' and substr(course_create_time,1,10) <= '%s'
#             and cast(course_amt as double)<50
#             and fk_tea_teacher in ('89254','89340','89309','89290','89334','89333','89307','89404')
#     ) a
#     left join
#     (
#         select 
#             stu_course_id 
#             ,hotspot_id
#         from wx_dwddb.dwd_mkt_order_course_channel_fd
#         where dt=date_format(date_add(current_date,-1),'yyyy-MM-dd')
#     ) b on b.stu_course_id = a.stu_course_id
#     group by 
#         b.hotspot_id
#         ,a.stu_course_id
#         ,a.fk_tea_teacher
# ) b on a.fk_source=b.hotspot_id

# """ % (start_yes,end_yes,start_yes_fmt,end_yes_fmt)

# result = spark.sql(sql)

# data_leads_chuangxin=result.toPandas()
# data_leads_chuangxin.head()


In [84]:
# BOXleads
sql = """
select
    'BOX' as `业务分类`
    ,count(distinct b.stu_course_id) as `购买人次`
from
(
    select 
        'BOX' as `业务分类`,
        fk_source
    from wx_adsdb.ads_mkt_bdp_app_mkt_adserving_summary_bdp_new_di_id  t1
    where 
        flag IN ('6')
        -- AND dt >= date_format(date_add(current_date,-45),'yyyyMMdd')
        -- AND dt <= date_format(date_add(current_date,-39),'yyyyMMdd')
        and dt >= '%s' AND dt <= '%s'
        and adver_id not in ('105855','105856','105857','105917','105925','105957','106005','106023','106142','106000','106004','106143','106159','106213','106199','106198')
        AND (client_type_new IN ('app','') or client_type_new is null)
        and line_name in ('大师课','素养课')
    group by
        'BOX',
        fk_source
) a
left join
(
    select
        b.hotspot_id
        ,a.stu_course_id
    from
    (
        select 
            replace(substr(course_create_time,1,10),'-','') as `dt`
            ,stu_course_id
            ,source_id
            ,buy_payed_amt
            ,course_amt
            ,fk_student
        from wx_dwsdb.dws_trd_sales_stucou_tag_all
        where 
            dt=date_format(date_add(current_date,-1),'yyyyMMdd')
            -- and type_1_id='3045'
            -- and course_type_first_desc in ('素养课','短期班')
            and is_return='0'
            -- and current_course_name='素养课-低价拼团'
            -- and substr(course_create_time,1,10) >=date_format(date_add(current_date,-44),'yyyy-MM-dd')
            -- and substr(course_create_time,1,10) <=date_format(date_add(current_date,-38),'yyyy-MM-dd')
            and substr(course_create_time,1,10)>='%s' and substr(course_create_time,1,10)<='%s'
    ) a
    left join
    (
        select 
            stu_course_id 
            ,hotspot_id
        from wx_dwddb.dwd_mkt_order_course_channel_fd
        where dt=date_format(date_add(current_date,-1),'yyyy-MM-dd')
    ) b on b.stu_course_id = a.stu_course_id
    group by 
        b.hotspot_id
        ,a.stu_course_id
) b on a.fk_source=b.hotspot_id
group by 
    'BOX'

""" % (start_yes,end_yes,start_yes_fmt,end_yes_fmt)

result = spark.sql(sql)

data_leads_box=result.toPandas()
data_leads_box.head()


,业务分类,购买人次
0,BOX,1278


In [85]:
# 图书leads
sql = """
select
    '图书' as `业务分类`
    ,count(distinct order_sn) as `购买人次`
    -- ,count(distinct user_id) as "人数"
    -- ,sum(order_price/100) as "GMV"

from ic_odsdb.ods_ms_xes_course_plat_xes_order_4_fd
where
    dt=date_format(date_add(current_date,-1),'yyyy-MM-dd')
    -- and order_status in (1,3)
    and substr(date_format(from_utc_timestamp(from_unixtime(order_pay_time),'GMT+8'),'yyyy-MM-dd HH:mm:ss'),1,10)>='%s'
    and substr(date_format(from_utc_timestamp(from_unixtime(order_pay_time),'GMT+8'),'yyyy-MM-dd HH:mm:ss'),1,10)<='%s'
group by
    '图书'

""" % (start_yes_fmt,end_yes_fmt)

result = spark.sql(sql)

data_leads_tushu=result.toPandas()
data_leads_tushu.head()


,业务分类,购买人次
0,图书,387


In [86]:
# 全真leads
sql = """
select
    '全真' as `业务分类`
    ,count(distinct b.`短期班订单ID`) as `购买人次`
from
    (
        select

            fk_source

        from wx_adsdb.ads_mkt_bdp_app_mkt_adserving_summary_bdp_new_di_id

        where 

            flag IN ('6')

            AND (client_type_new IN ('app','') or client_type_new is null)

            -- AND dt >= date_format(date_add(current_date,-45),'yyyyMMdd')

            -- AND dt <= date_format(date_add(current_date,-39),'yyyyMMdd')

            and dt>='%s' and dt<='%s'

            and adver_id not in ('105855','105856','105857','105917','105925','105957','106005','106023','106142','106000','106004','106143','106159','106213','106199','106198')

            and (line_name in ('口才','素材','next-科学','情商','增长','外部大V') or specific_name in ('公益活动'))

            and page_name='首页'

            and adslot_id in ('99','98','97','96','95','94','93','92','91','90','81','80','79','78','77','76','68','46','64','57','58','56','62','59','60')

        group by

            fk_source

    ) as a
left join
    (
        select
            b.dt as `统计截止日期`,
            a.course_id as `课程ID`,
            b.course_name as `课程名称`,
            a.goods_id as `商品ID`,
            b.work_type as `工作室ID`,
            b.course_type as `类型`,
            c.stu_id as `学生ID`,
            c.stu_course_id as stu_course_id,
            d.order_sn as `短期班订单ID`,
            d.price as `课程售价`,
            d.paid_time as `购短期课时间`,
            substr(d.paid_time, 1, 10) as `购短期班日期`,
            d.real_price as `实付价格`,
            d.source_id
        from
            (
            select
                cast(course_id as string) as course_id,
                cast(goods_id as string) as goods_id
            from
                wx_odsdb.ods_ms_next_app_nscm_goods_course_plans_fd 
            where
                dt = date_format(date_add(current_date,-1),'yyyy-MM-dd')
                and cast(goods_id as string) not in ('383054','380956','380752','382304','381672','379632','381585','381673','379496','380957','380958','379380','384285','383055','383071','381673','379621','380632','380642','379622','379620','379604','379578','379405','379404', '379352', '379371', '379381', '379446', '379486', '379558', '381695', '381821', '381687', '379605', '379479', '379480', '7000000', '7000003', '7000004', '7000006', '7000007', '7000016', '7000017', '7000021', '379588', '7000031', '7000033'
                                                        , '7000008', '7000000', '7000003', '7000032', '7000030')
            group by 
                cast(course_id as string),
                cast(goods_id as string)
            ) as a
        left join 
            (
            select
                dt,
                cast(id as string) as course_id,   -- 课程ID
                name as course_name,  -- 课程名称
                cast(course_studio_id as string) as work_type,   -- 工作室ID
                cast(course_mold as string) as course_type  -- 长短期班
            from
                wx_odsdb.ods_ms_next_app_nscm_courses_fd 
            where
                dt = date_format(date_add(current_date,-1),'yyyy-MM-dd')
                and cast(course_mold as string) in ('2')  -- 1 长期班   2 短期班
        -- 		and cast(course_studio_id as string) in ('115', '116', '117')
            ) as b on a.course_id = b.course_id
        left join 
            (
            select
                dt,
                cast(id as string) as stu_course_id,
                cast(course_id as string) as course_id,    -- 课程ID
                cast(goods_id as string) as goods_id,
                course_name,    -- 课程名称
                cast(course_studio_id as string) as work_type,  -- 工作室ID
                cast(stu_id as string) as stu_id,    -- 学生ID
                order_sn,    -- 订单ID
                cast(status as string) as status     -- 10 正常  20 已退费
            from
                wx_odsdb.ods_ms_next_app_nscm_stu_courses_fd
            where
                dt = date_format(date_add(current_date,-1),'yyyy-MM-dd')
                and cast(goods_id as string) not in ('383054','380956','380752','382304','381672','379632','381585','381673','379496','380957','380958','379380','384285','383055','383071','381673','379621','380632','380642','379622','379620','379604','379578','379405','379404', '379352', '379371', '379381', '379446', '379486', '379558', '381695', '381821', '381687', '379605', '379479', '379480', '7000000', '7000003', '7000004', '7000006', '7000007', '7000016', '7000017', '7000021', '379588', '7000031', '7000033'
                                                        , '7000008', '7000000', '7000003', '7000032', '7000030')
                and status = 10  --- 未退费
            ) as c on a.course_id = c.course_id and a.goods_id = c.goods_id and c.course_name = b.course_name and b.work_type = c.work_type and b.dt = c.dt
        left join 
            (
            select
                dt,
                order_sn,
                price,
                real_price,
                cast(stu_id as string) as stu_id,
                date_format(from_unixtime(paid_time), 'yyyy-MM-dd HH:mm:ss') as paid_time,
                source_id
            from
                wx_odsdb.ods_ms_next_app_nscm_orders_fd 
            where
                dt = date_format(date_add(current_date,-1),'yyyy-MM-dd')
                and pay_status = 20   -- 20 支付成功
                and price >= 0
                and substr(date_format(from_utc_timestamp(from_unixtime(paid_time),'GMT+8'),'yyyy-MM-dd'),1,10)>='%s'
                and substr(date_format(from_utc_timestamp(from_unixtime(paid_time),'GMT+8'),'yyyy-MM-dd'),1,10)<='%s'
                -- 添加购课时间 !!
                -- 添加购课金额 !!
            ) as d on c.order_sn = d.order_sn and c.stu_id = d.stu_id and c.dt = d.dt
    ) as b on a.fk_source = b.source_id
group by '全真'

""" % (start_yes,end_yes,start_yes_fmt,end_yes_fmt)

result = spark.sql(sql)

data_leads_next=result.toPandas()
data_leads_next.head()


,业务分类,购买人次
0,全真,243


### 转化率

In [87]:
# 全真转化率
sql = """
select
    '全真' as `业务分类`,
    count(b.`学生ID`)/count(a.`学生ID`) as `转化率`
from
    (
    select
        b.work_type as `工作室ID`,
        c.stu_id as `学生ID`
    from
        (
        select
            cast(course_id as string) as course_id,
            cast(goods_id as string) as goods_id
        from
            wx_odsdb.ods_ms_next_app_nscm_goods_course_plans_fd 
        where
            dt = date_format(date_add(current_date,-1),'yyyy-MM-dd')
            and cast(goods_id as string) not in ('383054','380956','380752','382304','381672','379632','381585','381673','379496','380957','380958','379380','384285','383055','383071','381673','379621','380632','380642','379622','379620','379604','379578','379405','379404', '379352', '379371', '379381', '379446', '379486', '379558', '381695', '381821', '381687', '379605', '379479', '379480', '7000000', '7000003', '7000004', '7000006', '7000007', '7000016', '7000017', '7000021', '379588', '7000031', '7000033'
                                                    , '7000008', '7000000', '7000003', '7000032', '7000030')
        group by 
            cast(course_id as string),
            cast(goods_id as string)
        ) as a
    left join 
        (
        select
            dt,
            cast(id as string) as course_id,   -- 课程ID
            name as course_name,  -- 课程名称
            cast(course_studio_id as string) as work_type,   -- 工作室ID
            cast(course_mold as string) as course_type  -- 长短期班
        from
            wx_odsdb.ods_ms_next_app_nscm_courses_fd 
        where
            dt = date_format(date_add(current_date,-1),'yyyy-MM-dd')    		
            and cast(course_mold as string) in ('2')  -- 1 长期班   2 短期班
            and cast(course_studio_id as string) in ('115', '116', '117')
        ) as b on a.course_id = b.course_id
    left join 
        (
        select
            dt,
            cast(course_id as string) as course_id,    -- 课程ID
            cast(goods_id as string) as goods_id,
            course_name,    -- 课程名称
            cast(course_studio_id as string) as work_type,  -- 工作室ID
            cast(stu_id as string) as stu_id,    -- 学生ID
            order_sn    -- 订单ID
        from
            wx_odsdb.ods_ms_next_app_nscm_stu_courses_fd
        where
            dt = date_format(date_add(current_date,-1),'yyyy-MM-dd') 
            and status = 10  --- 未退费
            and substr(date_format(from_utc_timestamp(from_unixtime(course_end_time),'GMT+8'),'yyyy-MM-dd'),1,10)<=date_format(date_add(current_date,-0),'yyyy-MM-dd') 
        ) as c on a.course_id = c.course_id and a.goods_id = c.goods_id and c.course_name = b.course_name and b.work_type = c.work_type and b.dt = c.dt
    inner join 
        (
        select
            dt,
            order_sn,
            price,
            real_price,
            cast(stu_id as string) as stu_id,
            order_type,
            source_id
        from
            wx_odsdb.ods_ms_next_app_nscm_orders_fd 
        where
            dt = date_format(date_add(current_date,-1),'yyyy-MM-dd') 
            and pay_status = 20   -- 20 支付成功
            and substr(date_format(from_utc_timestamp(from_unixtime(paid_time),'GMT+8'),'yyyy-MM-dd'),1,10)>='%s'
            and substr(date_format(from_utc_timestamp(from_unixtime(paid_time),'GMT+8'),'yyyy-MM-dd'),1,10)<='%s'
        ) as d on c.order_sn = d.order_sn and c.stu_id = d.stu_id and c.dt = d.dt
    where
        c.stu_id != ''
    ) as a 
left join 
    (
    select
        b.work_type as `工作室ID`,
        c.stu_id as `学生ID`
    from
        (
        select
            cast(course_id as string) as course_id,
            cast(goods_id as string) as goods_id
        from
            wx_odsdb.ods_ms_next_app_nscm_goods_course_plans_fd 
        where
            dt = date_format(date_add(current_date,-1),'yyyy-MM-dd')
            and cast(goods_id as string) not in ('383054','380956','380752','382304','381672','379632','381585','381673','379496','380957','380958','379380','384285','383055','383071','381673','379621','380632','380642','379622','379620','379604','379578','379405','379404', '379352', '379371', '379381', '379446', '379486', '379558', '381695', '381821', '381687', '379605', '379479', '379480', '7000000', '7000003', '7000004', '7000006', '7000007', '7000016', '7000017', '7000021', '379588', '7000031', '7000033'
                                                    , '7000008', '7000000', '7000003', '7000032', '7000030')
        group by 
            cast(course_id as string),
            cast(goods_id as string)
        ) as a
    left join 
        (
        select
            dt,
            cast(id as string) as course_id,   -- 课程ID
            name as course_name,  -- 课程名称
            cast(course_studio_id as string) as work_type,   -- 工作室ID
            cast(course_mold as string) as course_type  -- 长短期班
        from
            wx_odsdb.ods_ms_next_app_nscm_courses_fd 
        where
            dt = date_format(date_add(current_date,-1),'yyyy-MM-dd')    		
            and cast(course_mold as string) in ('1')  -- 1 长期班   2 短期班
            and cast(course_studio_id as string) in ('115', '116', '117')
        ) as b on a.course_id = b.course_id
    inner join 
        (
        select
            dt,
            cast(course_id as string) as course_id,    -- 课程ID
            cast(goods_id as string) as goods_id,
            course_name,    -- 课程名称
            cast(course_studio_id as string) as work_type,  -- 工作室ID
            cast(stu_id as string) as stu_id,    -- 学生ID
            order_sn    -- 订单ID
        from
            wx_odsdb.ods_ms_next_app_nscm_stu_courses_fd
        where
            dt = date_format(date_add(current_date,-1),'yyyy-MM-dd') 
            and status = 10  --- 未退费
            
        ) as c on a.course_id = c.course_id and a.goods_id = c.goods_id and c.course_name = b.course_name and b.work_type = c.work_type and b.dt = c.dt
    inner join 
        (
        select
            dt,
            order_sn,
            price,
            real_price,
            cast(stu_id as string) as stu_id,
            order_type,
            source_id
        from
            wx_odsdb.ods_ms_next_app_nscm_orders_fd 
        where
            dt = date_format(date_add(current_date,-1),'yyyy-MM-dd') 
            and pay_status = 20   -- 20 支付成功
        ) as d on c.order_sn = d.order_sn and c.stu_id = d.stu_id and c.dt = d.dt
    where
        c.stu_id != ''
    ) as b on a.`学生ID` = b.`学生ID` and a.`工作室ID` = b.`工作室ID`
group by 
    '全真'

""" % (end_yes_14_fmt,end_yes_fmt)

result = spark.sql(sql)

data_zhuanhualv_next=result.toPandas()
data_zhuanhualv_next.head()


,业务分类,转化率
0,全真,0.050486


In [88]:
# 素质导流课转化率
sql = """
SELECT
  t1.subject_names as `业务分类`,
  (sum(t1.same_subject_cnt) / sum(t1.div_cnt)) AS `转化率`
FROM
  wx_adsdb.ads_mkt_insite_sales_core_index_fd t1
WHERE
  dt=date_format(date_add(current_date,-1),'yyyy-MM-dd')
  and t1.course_create_date >= '%s'
  AND t1.course_create_date <= '%s'
  -- AND t1.course_create_date >='' and t1.course_create_date <= ''
  AND t1.division_name IN ('素质')
  and t1.order_second_channel_name='站内资源位'
  and t1.course_type_second_desc='爆品课'
  and adgroup_name not in ('网校APP-爆品课新人礼','网校APP-优选课新人礼','服务页顶部导航','网校APP-APP弹窗','touch端轮播图'
  ,'推广位','网校APP-支付完成页','网校APP-搜索区域-搜索热搜榜','站内客服','服务页中部列表','站内push','网校APP-搜索区域-搜索发现词'
  ,'网校APP-球区二级页导流课卡','网校APP-学习中心顶部运营位','push渠道','网校APP-搜索区域-轮播热搜词','0831奇妙地球任我拍'
  ,'编程36切0测试','科学课-29元','9元课拦截','我的页面底部banner','编程课-0元','续报扩科活动')
GROUP BY
  t1.subject_names

""" % (end_yes_14_fmt,end_yes_7_fmt)

result = spark.sql(sql)

data_zhuanhualv_suzhi=result.toPandas()
data_zhuanhualv_suzhi.head()


,业务分类,转化率
0,演讲与口才,0.043526
1,编程,0.019739
2,科学创想,0.025020
3,国际象棋,0.025188
4,人文美育,0.077181


In [89]:
# 老带新转化率
sql = """
SELECT
  case when t1.site_id='6475' then '常规老带新' else '特训班老带新' end as `业务分类`,
  (
    sum(
      if(
        is_valid = '1'
        and n_is_return = '0'
        and is_return = '0',
        cast(ifNull(normal_pt, '0') as double),
        0
      )
    ) / count(distinct if(is_return = '0', stu_course_id, null))
  ) AS `转化率`
FROM
  wx_adsdb.ads_mkt_trans_joint_fd t1
WHERE
  dt=date_format(date_add(current_date,-1),'yyyy-MM-dd')
  and t1.site_id IN ('6879','6475')
  AND substr(t1.course_create_time,1,10) >= '%s'
  AND substr(t1.course_create_time,1,10) <= '%s'
group by 
    case when t1.site_id='6475' then '常规老带新' else '特训班老带新' end

""" % (end_yes_14_fmt,end_yes_7_fmt)

result = spark.sql(sql)

data_zhuanhualv_laodaixin=result.toPandas()
data_zhuanhualv_laodaixin.head()


,业务分类,转化率
0,常规老带新,0.052512
1,特训班老带新,0.009407


In [90]:
# 创新转化率
sql = """
select
    case when a.fk_tea_teacher in ('89254','89340','89182') then "自然博物"
         when a.fk_tea_teacher in ('89309') then "航天"
         when a.fk_tea_teacher in ('89290') then "ART" 
         when a.fk_tea_teacher in ('89334','89333','89307','89404') then "TED"
         else "其他" end as `业务分类`
    ,count(b.stu_course_id)/count(a.stu_course_id) as `转化率`
from
(
    select 
        is_return,
        return_time,
        dt, --  as "统计截止日期",
        fk_tea_teacher,
        fk_course,
        fk_tea_counselor,
        counselor_name,
        course_end_time,
        course_start_time,
        course_create_time,
        fk_student,
        buy_payed_amt,
        stu_course_id,
        current_course_name
    from wx_dwsdb.dws_trd_sales_stucou_tag_all
    where 
        dt=date_format(date_add(current_date,-1),'yyyyMMdd')
        -- and type_1_id='3045'
        and course_amt<50
        AND substr(course_create_time,1,10) >= '%s'
        AND substr(course_create_time,1,10) <= '%s'
        and substr(course_end_time,1,10) <= '%s'
        and is_return='0'
        and fk_tea_teacher in ('89254','89340' ,'89182','89309','89290','89334','89333','89307','89404')
    -- group by
    --     fk_tea_teacher
) a
left join
(
    select 
        stu_course_id 
        ,hotspot_id
        ,order_first_channel_name
        ,order_second_channel_name
    from wx_dwddb.dwd_mkt_order_course_channel_fd
    where dt=date_format(date_add(current_date,-1),'yyyy-MM-dd')
) c on c.stu_course_id = a.stu_course_id
left join
(
    select 
        is_return,
        dt, --  as "统计截止日期",
        fk_course,
        course_end_time,
        course_start_time,
        current_course_name, -- as "课程名称",
        course_create_time,
        fk_student,
        buy_payed_amt,
        stu_course_id,
        current_course_name
    from wx_dwsdb.dws_trd_sales_stucou_tag_all
    where 
        dt=date_format(date_add(current_date,-1),'yyyyMMdd')
        -- and type_1_id='3045'
        and course_amt>=400
        and is_return='0'
        and fk_tea_teacher in ('89254','89340','89182','89309','89290','89334','89333','89307','89404')
    -- group by
    --     fk_tea_teacher
) b on a.fk_student=b.fk_student
group by
    case when a.fk_tea_teacher in ('89254','89340','89182') then "自然博物"
         when a.fk_tea_teacher in ('89309') then "航天"
         when a.fk_tea_teacher in ('89290') then "ART" 
         when a.fk_tea_teacher in ('89334','89333','89307','89404') then "TED"
         else "其他" end

""" % (end_yes_14_fmt,end_yes_7_fmt,end_yes_fmt)

result = spark.sql(sql)

data_zhuanhualv_chuagnxin=result.toPandas()
data_zhuanhualv_chuagnxin.head()


,业务分类,转化率
0,TED,0.000000
1,ART,0.095960
2,自然博物,0.145286


### 客单价

In [91]:
# 素质导流课客单价
sql = """
select
    b.subject_names as `业务分类`
    ,(coalesce(sum(a.pay_amt),0)+coalesce(sum(c.real_pay_amt),0))/count(distinct a.stu_course_id) as `客单价`
from
(
   SELECT 
        *
    FROM wx_dwddb.dwd_mkt_trans_div_course_fd
    WHERE 
        dt = date_format(date_add(current_date,-1),'yyyy-MM-dd')
        AND order_return_time = '' --未退费
        AND substr(trans_sign,1,1) = '1' --三天转化
        and substr(order_pay_end_time,1,10) >= '%s'
        and substr(order_pay_end_time,1,10) <= '%s'
        AND is_valid = 1 --转化有效
) a
left join
(
    SELECT 
        *
    FROM wx_dwddb.dwd_trd_pro_order_fd
    WHERE 
        dt = date_format(date_add(current_date,-1),'yyyy-MM-dd')
        AND is_return='0'
        and substr(pay_end_time,1,10)>='%s'
        and substr(pay_end_time,1,10)<='%s'
) c on c.order_id=a.order_id
inner join
(
    SELECT 
        *
    FROM wx_dwddb.dwd_mkt_order_div_course_fd
    WHERE 
        dt=date_format(date_add(current_date,-1),'yyyy-MM-dd')
        AND subject_names IN ('人文美育','科学创想','编程','国际象棋','演讲与口才')
        AND order_return_time = '' --未退费
        and order_first_channel_name='站内'
) b on a.div_stu_course_id = b.stu_course_id
group by
    b.subject_names

""" % (end_yes_29_fmt,end_yes_fmt,end_yes_29_fmt,end_yes_fmt)

result = spark.sql(sql)

data_kedanjia_suzhi=result.toPandas()
data_kedanjia_suzhi.head()


,业务分类,客单价
0,演讲与口才,1241.624309
1,编程,2377.198966
2,科学创想,1512.719048
3,国际象棋,1391.943563
4,人文美育,1550.962758


In [92]:
# 老带新客单价
sql = """
select
    '老带新' as `大类`
    ,coalesce(sum(a.pay_amt),0)/count(distinct a.stu_course_id) as `客单价`
from
(
   SELECT 
        *
    FROM wx_dwddb.dwd_mkt_trans_div_course_fd
    WHERE 
        dt = date_format(date_add(current_date,-1),'yyyy-MM-dd')
        AND order_return_time = '' --未退费
        AND substr(trans_sign,1,1) = '1' --三天转化
        and substr(order_pay_end_time,1,10) >= '%s'
        and substr(order_pay_end_time,1,10) <= '%s'
        AND is_valid = 1 --转化有效
) a
inner join
(
    SELECT 
        *
    FROM wx_dwddb.dwd_mkt_order_div_course_fd
    WHERE 
        dt=date_format(date_add(current_date,-1),'yyyy-MM-dd')
        AND subject_names IN ('人文美育','科学创想','编程','国际象棋','演讲与口才')
        AND order_return_time = '' --未退费
        and order_first_channel_name='老带新'
) b on a.div_stu_course_id = b.stu_course_id
group by
    '老带新'

""" % (end_yes_29_fmt,end_yes_fmt)

result = spark.sql(sql)

data_kedanjia_laodaixin=result.toPandas()
data_kedanjia_laodaixin.head()


,大类,客单价
0,老带新,1497.218468


In [93]:
# 创新客单价
sql = """
select 
    case when fk_tea_teacher in ('89254','89340','89182') then "自然博物"
         when fk_tea_teacher in ('89309') then "航天"
         when fk_tea_teacher in ('89290') then "ART" 
         when fk_tea_teacher in ('89334','89333','89307','89404') then "TED"
         else "其他" end as `业务分类`
    ,sum(buy_payed_amt)/count(stu_course_id) as `客单价`
from wx_dwsdb.dws_trd_sales_stucou_tag_all
where 
    dt=date_format(date_add(current_date,-1),'yyyyMMdd')
    -- and type_1_id='3045'
    -- and course_amt>=400
    and course_type_second_desc='旧大师课'
    and substr(course_create_time,1,10)>='%s'
    and substr(course_create_time,1,10)<='%s'
    and is_return='0'
    and buy_payed_amt>0
    and fk_tea_teacher in ('89254','89340','89182','89309','89290','89334','89333','89307','89404')
group by
    case when fk_tea_teacher in ('89254','89340','89182') then "自然博物"
         when fk_tea_teacher in ('89309') then "航天"
         when fk_tea_teacher in ('89290') then "ART" 
         when fk_tea_teacher in ('89334','89333','89307','89404') then "TED"
         else "其他" end

""" % (end_yes_29_fmt,end_yes_fmt)

result = spark.sql(sql)

data_kedanjia_chuangxin=result.toPandas()
data_kedanjia_chuangxin.head()


,业务分类,客单价
0,ART,340.308057
1,自然博物,433.949027


In [94]:
# 全真单价
sql = """
select
    '全真' as `业务分类`
    ,sum(a.`实付价格`) * 1.0 / count(distinct a.`长期班订单ID`) * 0.01 as `客单价`
from
    (
        select distinct
            a.`长期班订单ID`,
            -- a.`课程售价`,
            a.`实付价格`

        from
            (
                select
                    b.dt as `统计截止日期`,
                    a.course_id as `课程ID`,
                    b.course_name as `课程名称`,
                    a.goods_id as `商品ID`,
                    b.work_type as `工作室ID`,
                    b.course_type as `类型`,
                    c.stu_id as `学生ID`,
                    c.stu_course_id as stu_course_id,
                    substr(cast(c.course_start_time as string), 1, 10) as `长期班开始时间`,
                    substr(cast(c.course_end_time as string), 1, 10) as `长期班结束时间`,
                    d.order_sn as `长期班订单ID`,
                    d.price as `课程售价`,
                    d.paid_time as `长期课支付时间`,
                    substr(d.paid_time, 1, 10) as `长期班支付日期`,
                    d.real_price as `实付价格`,
                    d.source_id
                from
                    (
                        select
                            cast(course_id as string) as course_id,
                            cast(goods_id as string) as goods_id
                        from
                            wx_odsdb.ods_ms_next_app_nscm_goods_course_plans_fd 
                        where
                            dt = date_format(date_add(current_date,-1),'yyyy-MM-dd')
                            and cast(goods_id as string) not in ('383054','380956','380752','382304','381672','379632','381585','381673','379496','380957','380958','379380','384285','383055','383071','381673','379621','380632','380642','379622','379620','379604','379578','379405','379404', '379352', '379371', '379381', '379446', '379486', '379558', '381695', '381821', '381687', '379605', '379479', '379480', '7000000', '7000003', '7000004', '7000006', '7000007', '7000016', '7000017', '7000021', '379588', '7000031', '7000033'
                                                                    , '7000008', '7000000', '7000003', '7000032', '7000030')
                        group by 
                            cast(course_id as string),
                            cast(goods_id as string)
                    ) as a
                left join 
                    (
                        select
                            dt,
                            cast(id as string) as course_id,   -- 课程ID
                            name as course_name,  -- 课程名称
                            cast(course_studio_id as string) as work_type,   -- 工作室ID
                            cast(course_mold as string) as course_type  -- 长短期班
                        from
                            wx_odsdb.ods_ms_next_app_nscm_courses_fd 
                        where
                            dt = date_format(date_add(current_date,-1),'yyyy-MM-dd')
                            and cast(course_mold as string) in ('1')  -- 1 长期班   2 短期班
                    ) as b on a.course_id = b.course_id
                left join 
                    (
                    select
                        dt,
                        cast(id as string) as stu_course_id,
                        cast(course_id as string) as course_id,    -- 课程ID
                        cast(goods_id as string) as goods_id,
                        course_name,    -- 课程名称
                        cast(course_studio_id as string) as work_type,  -- 工作室ID
                        cast(stu_id as string) as stu_id,    -- 学生ID
                        order_sn,    -- 订单ID
                        date_format(from_unixtime(course_start_time), 'yyyy-MM-dd HH:mm:ss') as course_start_time,   -- 课程开始时间
                        date_format(from_unixtime(course_end_time), 'yyyy-MM-dd HH:mm:ss') as course_end_time,    -- 课程结束时间
                        cast(status as string) as status     -- 10 正常  20 已退费
                    from
                        wx_odsdb.ods_ms_next_app_nscm_stu_courses_fd
                    where
                        dt = date_format(date_add(current_date,-1),'yyyy-MM-dd')
                        and cast(goods_id as string) not in ('383054','380956','380752','382304','381672','379632','381585','381673','379496','380957','380958','379380','384285','383055','383071','381673','379621','380632','380642','379622','379620','379604','379578','379405','379404', '379352', '379371', '379381', '379446', '379486', '379558', '381695', '381821', '381687', '379605', '379479', '379480', '7000000', '7000003', '7000004', '7000006', '7000007', '7000016', '7000017', '7000021', '379588', '7000031', '7000033'
                                                                , '7000008', '7000000', '7000003', '7000032', '7000030')
                        and status = 10  --- 未退费
                    ) as c on a.course_id = c.course_id and a.goods_id = c.goods_id and c.course_name = b.course_name and b.work_type = c.work_type and b.dt = c.dt
                left join 
                    (
                    select
                        dt,
                        order_sn,
                        price,
                        real_price,
                        cast(stu_id as string) as stu_id,
                        date_format(from_unixtime(paid_time), 'yyyy-MM-dd HH:mm:ss') as paid_time,
                        source_id
                    from
                        wx_odsdb.ods_ms_next_app_nscm_orders_fd 
                    where
                        dt = date_format(date_add(current_date,-1),'yyyy-MM-dd')
                        and pay_status = 20   -- 20 支付成功
                        and price >= 0
                        and substr(date_format(from_unixtime(paid_time), 'yyyy-MM-dd HH:mm:ss'), 1, 10) >= '%s'
                        and substr(date_format(from_unixtime(paid_time), 'yyyy-MM-dd HH:mm:ss'), 1, 10) <= '%s'
                        -- 添加购课时间 !!
                        -- 添加购课金额 !!
                    ) as d on c.order_sn = d.order_sn and c.stu_id = d.stu_id and c.dt = d.dt
            ) as a
        where   
            a.`学生ID` is not null
            and source_id is not null
    ) as a
group by
    '全真'
""" % (end_yes_29_fmt,end_yes_fmt)

result = spark.sql(sql)

data_kedanjia_next=result.toPandas()
data_kedanjia_next.head()


,业务分类,客单价
0,全真,554.798191489361702


### GMV直销

In [95]:
# BOXGMV
sql = """
select
    'BOX' as `业务分类`
    -- ,count(distinct stu_course_id)
    ,sum(b.buy_payed_amt) as `GMV直售`
from
(
    select 
        'BOX' as `业务分类`,
        fk_source
    from wx_adsdb.ads_mkt_bdp_app_mkt_adserving_summary_bdp_new_di_id  t1
    where 
        flag IN ('6')
        AND (client_type_new IN ('app','') or client_type_new is null)
        -- and dt>=date_format(date_add(current_date,-44),'yyyyMMdd')
        -- and dt<=date_format(date_add(current_date,-38),'yyyyMMdd')
        AND dt >= '%s' AND dt <= '%s'
        and adver_id not in ('105855','105856','105857','105917','105925','105957','106005','106023','106142','106000','106004','106143','106159','106213','106199','106198')
        and line_name in ('大师课','素养课')
    group by
        'BOX',
        fk_source
) a
left join
(
    select
        b.hotspot_id
        ,a.buy_payed_amt
        ,a.stu_course_id
    from
    (
        select 
            stu_course_id
            ,source_id
            ,buy_payed_amt
            ,course_amt
            ,fk_student
        from wx_dwsdb.dws_trd_sales_stucou_tag_all
        where 
            dt=date_format(date_add(current_date,-1),'yyyyMMdd')
            -- and type_1_id='3045'
            -- and course_type_first_desc in ('素养课','短期班')
            and is_return='0'
            -- and current_course_name='素养课-低价拼团'
            -- and substr(course_create_time,1,10) >=date_format(date_add(current_date,-44),'yyyy-MM-dd')
            -- and substr(course_create_time,1,10) <=date_format(date_add(current_date,-38),'yyyy-MM-dd')
            and substr(course_create_time,1,10)>='%s' and substr(course_create_time,1,10)<='%s'
    ) a
    left join
    (
        select 
            stu_course_id 
            ,hotspot_id
        from wx_dwddb.dwd_mkt_order_course_channel_fd
        where dt=date_format(date_add(current_date,-1),'yyyy-MM-dd')
    ) b on b.stu_course_id = a.stu_course_id
    group by 
        b.hotspot_id
        ,a.buy_payed_amt
        ,a.stu_course_id
) b on a.fk_source=b.hotspot_id
group by 
    'BOX'

""" % (start_yes,end_yes,start_yes_fmt,end_yes_fmt)

result = spark.sql(sql)

data_gmv1_box=result.toPandas()
data_gmv1_box.head()

,业务分类,GMV直售
0,BOX,154222.0


In [96]:
# 图书GMV
sql = """
select
    '图书' as `业务分类`
    -- ,count(distinct order_sn) as `购买人次`
    -- ,count(distinct user_id) as "人数"
    ,sum(order_price/100) as `GMV直售`

from ic_odsdb.ods_ms_xes_course_plat_xes_order_4_fd
where
    dt=date_format(date_add(current_date,-1),'yyyy-MM-dd')
    -- and order_status in (1,3)
    and substr(date_format(from_utc_timestamp(from_unixtime(order_pay_time),'GMT+8'),'yyyy-MM-dd'),1,10)>='%s'
    and substr(date_format(from_utc_timestamp(from_unixtime(order_pay_time),'GMT+8'),'yyyy-MM-dd'),1,10)<='%s'
group by
    '图书'

""" % (start_yes_fmt,end_yes_fmt)

result = spark.sql(sql)

data_gmv1_tushu=result.toPandas()
data_gmv1_tushu.head()


,业务分类,GMV直售
0,图书,98665.18


In [97]:
# 创新GMV
sql = """
select
    case when b.fk_tea_teacher in ('89254','89340','89182') then '自然博物' 
             when b.fk_tea_teacher in ('89309') then '航天'
             when b.fk_tea_teacher in ('89290') then 'ART' 
             when b.fk_tea_teacher in ('89334','89333','89307','89404') then 'TED'
             else '其他' end as `业务分类`
    ,count(distinct b.stu_course_id) as `购买人次`
    ,sum(b.buy_payed_amt) as `GMV直售`
from
(
    select 
        fk_source
    from wx_adsdb.ads_mkt_bdp_app_mkt_adserving_summary_bdp_new_di_id a
    where 
        flag IN ('6')
        AND (client_type_new IN ('app','') or client_type_new is null)
        -- AND dt >= date_format(date_add(current_date,-45),'yyyyMMdd')
        -- AND dt <= date_format(date_add(current_date,-39),'yyyyMMdd')
        and dt>='%s' and dt<='%s'
        and (line_name in ('ART','TED','航天','自然博物') or adver_id in ('105855','105856','105857','105917','105925','105957','106005','106023','106142','106000','106004','106143','106159','106213','106199','106198'))
        and page_name='首页'
        and adslot_id in ('99','98','97','96','95','94','93','92','91','90','81','80','79','78','77','76','68','46','64','57','58','56','62','59','60')
    group by
        fk_source
) a
left join
(
    select
        b.hotspot_id
        ,a.stu_course_id
        ,a.fk_tea_teacher
        ,buy_payed_amt
    from
    (
        select 
            distinct
            fk_tea_teacher
            ,stu_course_id
            ,buy_payed_amt
        from wx_dwsdb.dws_trd_sales_stucou_tag_all
        where 
            dt=date_format(date_add(current_date,-1),'yyyyMMdd')
            and is_return='0'
            -- and substr(course_create_time,1,10) >=date_format(date_add(current_date,-44),'yyyy-MM-dd')
            -- and substr(course_create_time,1,10) <=date_format(date_add(current_date,-38),'yyyy-MM-dd')
            and substr(course_create_time,1,10) >= '%s' and substr(course_create_time,1,10) <= '%s'
            and cast(course_amt as double)>400
            and fk_tea_teacher in ('89254','89340','89182','89309','89290','89334','89333','89307','89404')
    ) a
    inner join
    (
        select 
            distinct stu_course_id 
        from wx_tmpdb.chuangxin_zhengjiake_laiyuanbiao_v2 a
        where 
            substr(course_create_time,1,10) >= '%s'
            and substr(course_create_time,1,10) <= '%s'
            and is_return='0'
            -- and first_channel='站内'
            and type='直售'
            and dt='%s'
    ) c on a.stu_course_id=c.stu_course_id
    left join
    (
        select 
            stu_course_id 
            ,hotspot_id
        from wx_dwddb.dwd_mkt_order_course_channel_fd
        where dt=date_format(date_add(current_date,-1),'yyyy-MM-dd')
    ) b on b.stu_course_id = c.stu_course_id
    group by 
        b.hotspot_id
        ,a.stu_course_id
        ,a.fk_tea_teacher
        ,buy_payed_amt
) b on a.fk_source=b.hotspot_id
group by 
    case when b.fk_tea_teacher in ('89254','89340','89182') then '自然博物' 
             when b.fk_tea_teacher in ('89309') then '航天'
             when b.fk_tea_teacher in ('89290') then 'ART' 
             when b.fk_tea_teacher in ('89334','89333','89307','89404') then 'TED'
             else '其他' end

""" % (start_yes,end_yes,start_yes_fmt,end_yes_fmt,start_yes_fmt,end_yes_fmt,end_yes_fmt)

result = spark.sql(sql)

data_gmv1_chuangxin=result.toPandas()
data_gmv1_chuangxin.head()


,业务分类,购买人次,GMV直售
0,自然博物,17,7713.0
1,其他,0,NaN


In [98]:
# 全真GMV
sql = """
select
    '全真' as `业务分类`
    ,count(distinct b.`长期班订单ID`) as `购买人次`
    ,sum(b.`实付价格`) as `GMV直售`
from
    (
        select

            fk_source

        from wx_adsdb.ads_mkt_bdp_app_mkt_adserving_summary_bdp_new_di_id

        where 

            flag IN ('6')

            AND (client_type_new IN ('app','') or client_type_new is null)

            -- AND dt >= date_format(date_add(current_date,-45),'yyyyMMdd')

            -- AND dt <= date_format(date_add(current_date,-39),'yyyyMMdd')

            and dt>='20220701' and dt<='20220731'

            and adver_id not in ('105855','105856','105857','105917','105925','105957','106005','106023','106142','106000','106004','106143','106159','106213','106199','106198')

            and (line_name in ('口才','素材','next-科学','情商','增长','外部大V') or specific_name in ('公益活动'))

            and page_name='首页'

            -- and adslot_id in ('99','98','97','96','95','94','93','92','91','90','81','80','79','78','77','76','68','46','64','57','58','56','62','59','60')

        group by

            fk_source

    ) as a
left join
    (
        select distinct
            a.`长期班订单ID`,
            -- a.`课程售价`,
            a.`实付价格`,
            a.source_id
        from
            (
                select
                    b.dt as `统计截止日期`,
                    a.course_id as `课程ID`,
                    b.course_name as `课程名称`,
                    a.goods_id as `商品ID`,
                    b.work_type as `工作室ID`,
                    b.course_type as `类型`,
                    c.stu_id as `学生ID`,
                    c.stu_course_id as stu_course_id,
                    substr(cast(c.course_start_time as string), 1, 10) as `长期班开始时间`,
                    substr(cast(c.course_end_time as string), 1, 10) as `长期班结束时间`,
                    d.order_sn as `长期班订单ID`,
                    d.price as `课程售价`,
                    d.paid_time as `长期课支付时间`,
                    substr(d.paid_time, 1, 10) as `长期班支付日期`,
                    d.real_price as `实付价格`,
                    d.source_id
                from
                    (
                        select
                            cast(course_id as string) as course_id,
                            cast(goods_id as string) as goods_id
                        from
                            wx_odsdb.ods_ms_next_app_nscm_goods_course_plans_fd 
                        where
                            dt = date_format(date_add(current_date,-1),'yyyy-MM-dd')
                            and cast(goods_id as string) not in ('383054','380956','380752','382304','381672','379632','381585','381673','379496','380957','380958','379380','384285','383055','383071','381673','379621','380632','380642','379622','379620','379604','379578','379405','379404', '379352', '379371', '379381', '379446', '379486', '379558', '381695', '381821', '381687', '379605', '379479', '379480', '7000000', '7000003', '7000004', '7000006', '7000007', '7000016', '7000017', '7000021', '379588', '7000031', '7000033'
                                                                    , '7000008', '7000000', '7000003', '7000032', '7000030')
                        group by 
                            cast(course_id as string),
                            cast(goods_id as string)
                    ) as a
                left join 
                    (
                        select
                            dt,
                            cast(id as string) as course_id,   -- 课程ID
                            name as course_name,  -- 课程名称
                            cast(course_studio_id as string) as work_type,   -- 工作室ID
                            cast(course_mold as string) as course_type  -- 长短期班
                        from
                            wx_odsdb.ods_ms_next_app_nscm_courses_fd 
                        where
                            dt = date_format(date_add(current_date,-1),'yyyy-MM-dd')
                            and cast(course_mold as string) in ('1')  -- 1 长期班   2 短期班
                    ) as b on a.course_id = b.course_id
                left join 
                    (
                    select
                        dt,
                        cast(id as string) as stu_course_id,
                        cast(course_id as string) as course_id,    -- 课程ID
                        cast(goods_id as string) as goods_id,
                        course_name,    -- 课程名称
                        cast(course_studio_id as string) as work_type,  -- 工作室ID
                        cast(stu_id as string) as stu_id,    -- 学生ID
                        order_sn,    -- 订单ID
                        date_format(from_unixtime(course_start_time), 'yyyy-MM-dd HH:mm:ss') as course_start_time,   -- 课程开始时间
                        date_format(from_unixtime(course_end_time), 'yyyy-MM-dd HH:mm:ss') as course_end_time,    -- 课程结束时间
                        cast(status as string) as status     -- 10 正常  20 已退费
                    from
                        wx_odsdb.ods_ms_next_app_nscm_stu_courses_fd
                    where
                        dt = date_format(date_add(current_date,-1),'yyyy-MM-dd')
                        and cast(goods_id as string) not in ('383054','380956','380752','382304','381672','379632','381585','381673','379496','380957','380958','379380','384285','383055','383071','381673','379621','380632','380642','379622','379620','379604','379578','379405','379404', '379352', '379371', '379381', '379446', '379486', '379558', '381695', '381821', '381687', '379605', '379479', '379480', '7000000', '7000003', '7000004', '7000006', '7000007', '7000016', '7000017', '7000021', '379588', '7000031', '7000033'
                                                                , '7000008', '7000000', '7000003', '7000032', '7000030')
                        and status = 10  --- 未退费
                    ) as c on a.course_id = c.course_id and a.goods_id = c.goods_id and c.course_name = b.course_name and b.work_type = c.work_type and b.dt = c.dt
                left join 
                    (
                    select
                        dt,
                        order_sn,
                        price,
                        real_price,
                        cast(stu_id as string) as stu_id,
                        date_format(from_unixtime(paid_time), 'yyyy-MM-dd HH:mm:ss') as paid_time,
                        source_id
                    from
                        wx_odsdb.ods_ms_next_app_nscm_orders_fd 
                    where
                        dt = date_format(date_add(current_date,-1),'yyyy-MM-dd')
                        and pay_status = 20   -- 20 支付成功
                        and price >= 0
                        and substr(date_format(from_unixtime(paid_time), 'yyyy-MM-dd HH:mm:ss'), 1, 10) >= '2022-07-01'
                        and substr(date_format(from_unixtime(paid_time), 'yyyy-MM-dd HH:mm:ss'), 1, 10) <= '2022-07-31'
                        -- 添加购课时间 !!
                        -- 添加购课金额 !!
                    ) as d on c.order_sn = d.order_sn and c.stu_id = d.stu_id and c.dt = d.dt
            ) as a
        where   
            a.`学生ID` is not null
            and source_id is not null
    ) as b on a.fk_source = b.source_id
group by '全真'

""" # % (start_yes,end_yes,start_yes_fmt,end_yes_fmt)

result = spark.sql(sql)

data_gmv1_next=result.toPandas()
data_gmv1_next.head()


,业务分类,购买人次,GMV直售
0,全真,0,None


### 汇总

In [ ]:
# BOX
# 首页日志
# pd.set_option('display.float_format',lambda x : '%.0f' % x)
data_rihuo_box=data_rihuo.copy()
data_rihuo_box['业务分类']='BOX'
data_rihuo_box=data_rihuo_box[['业务分类','累计日活']]
data_rihuo_box['累计日活']=data_rihuo_box['累计日活'].astype('int')

# 曝光UV
data_baoguang_box_1=data_baoguan_box.groupby('业务分类')['曝光UV'].agg(['sum']).reset_index()
data_baoguang_box_1.columns=['业务分类','首页资源位曝光UV']
data_baoguang_box_1['首页资源位曝光UV']=data_baoguang_box_1['首页资源位曝光UV'].astype('int')

data_box=pd.merge(data_rihuo_box,data_baoguang_box_1,on='业务分类',how='left')

# 购买人次/leads
data_leads_box_1=data_leads_box.copy()
data_leads_box_1.columns=['业务分类','购买人次/leads']
data_leads_box_1['购买人次/leads']=data_leads_box_1['购买人次/leads'].astype('int')

data_box=pd.merge(data_box,data_leads_box_1,on='业务分类',how='left')

# 转化率
data_zhuanhualv_box_1=data_baoguang_box_1.copy()
data_zhuanhualv_box_1['转化率']=1
data_zhuanhualv_box_1.drop(columns='首页资源位曝光UV',inplace=True)
data_zhuanhualv_box_1['转化率']=data_zhuanhualv_box_1['转化率'].astype('float')

data_box=pd.merge(data_box,data_zhuanhualv_box_1,on='业务分类',how='left')

# 曝光渗透率
data_shentoulv_box_1=data_baoguang_box_1.copy()
data_shentoulv_box_1['曝光渗透率']=data_baoguang_box_1['首页资源位曝光UV']/data_rihuo_box['累计日活']
data_shentoulv_box_1.drop(columns='首页资源位曝光UV',inplace=True)
data_shentoulv_box_1['曝光渗透率']=data_shentoulv_box_1['曝光渗透率'].astype('float')

data_box=pd.merge(data_box,data_shentoulv_box_1,on='业务分类',how='left')

# 曝光购买率
data_goumailv_box_1=data_leads_box_1.copy()
data_goumailv_box_1['曝光购买率']=data_leads_box_1['购买人次/leads']/data_baoguang_box_1['首页资源位曝光UV']
data_goumailv_box_1.drop(columns='购买人次/leads',inplace=True)
data_goumailv_box_1['曝光购买率']=data_goumailv_box_1['曝光购买率'].astype('float')

data_box=pd.merge(data_box,data_goumailv_box_1,on='业务分类',how='left')

# 客单价
data_kedanjia_box_1=data_baoguang_box_1.copy()
data_kedanjia_box_1['客单价']=0
data_kedanjia_box_1.drop(columns='首页资源位曝光UV',inplace=True)
data_kedanjia_box_1['客单价']=data_kedanjia_box_1['客单价'].astype('int')

data_box=pd.merge(data_box,data_kedanjia_box_1,on='业务分类',how='left')

# GMV转化
data_gmv_box=data_baoguang_box_1.copy()
data_gmv_box['GMV转化']=0
data_gmv_box.drop(columns='首页资源位曝光UV',inplace=True)

data_box=pd.merge(data_box,data_gmv_box,on='业务分类',how='left')

# GMV直售
# data_gmv1_box

data_box=pd.merge(data_box,data_gmv1_box,on='业务分类',how='left')

# GMV总计
data_box['GMV总计']=data_box['GMV转化']+data_box['GMV直售']

# 变现效率
data_box['变现效率']=data_box['GMV总计']/data_box['首页资源位曝光UV']

# 汇总
# data_box=pd.merge(data_box,data_bianxian_box_1,on='业务分类')
data_box_all=data_box.copy()
data_box_all['大类']='BOX'
data_box_all['日期']=start_date+'-'+end_date
data_box_all=data_box_all[['日期','大类','业务分类', '累计日活', '首页资源位曝光UV', '购买人次/leads', '转化率', '曝光渗透率', '曝光购买率','客单价', 'GMV转化', 'GMV直售','GMV总计','变现效率']]
data_box_all['客单价']=data_box_all['GMV直售']/data_box_all['购买人次/leads']
data_box_all['客单价']=data_box_all['客单价'].astype('float').astype('int')
data_box_all


In [ ]:
# 全真
# 首页日志
data_rihuo_next=data_rihuo.copy()
data_rihuo_next['业务分类']='全真'
data_rihuo_next=data_rihuo_next[['业务分类','累计日活']]
data_rihuo_next['累计日活']=data_rihuo_next['累计日活'].astype('int')

# 曝光UV
data_baoguang_next_1=data_baoguang_next.groupby('业务分类')['曝光UV'].agg(['sum']).reset_index()
data_baoguang_next_1.columns=['业务分类','首页资源位曝光UV']
data_baoguang_next_1['首页资源位曝光UV']=data_baoguang_next_1['首页资源位曝光UV'].astype('int')

data_next=pd.merge(data_rihuo_next,data_baoguang_next_1,on='业务分类',how='left')

# 购买人次/leads
data_leads_next_1=data_leads_next.copy()
data_leads_next_1.columns=['业务分类','购买人次/leads']
data_leads_next_1['购买人次/leads']=data_leads_next_1['购买人次/leads'].astype('int')

data_next=pd.merge(data_next,data_leads_next_1,on='业务分类',how='left')

# 转化率
data_zhuanhualv_next_1=data_zhuanhualv_next.copy()
data_zhuanhualv_next_1['转化率']=data_zhuanhualv_next_1['转化率'].astype('float')

data_next=pd.merge(data_next,data_zhuanhualv_next_1,on='业务分类',how='left')

# 曝光渗透率
data_shentoulv_next_1=data_baoguang_next_1.copy()
data_shentoulv_next_1['曝光渗透率']=data_baoguang_next_1['首页资源位曝光UV']/data_rihuo_next['累计日活']
data_shentoulv_next_1.drop(columns='首页资源位曝光UV',inplace=True)
data_shentoulv_next_1['曝光渗透率']=data_shentoulv_next_1['曝光渗透率'].astype('float')

data_next=pd.merge(data_next,data_shentoulv_next_1,on='业务分类',how='left')

# 曝光购买率
data_goumailv_next_1=data_leads_next_1.copy()
data_goumailv_next_1['曝光购买率']=data_leads_next_1['购买人次/leads']/data_baoguang_next_1['首页资源位曝光UV']
data_goumailv_next_1.drop(columns='购买人次/leads',inplace=True)
data_goumailv_next_1['曝光购买率']=data_goumailv_next_1['曝光购买率'].astype('float')

data_next=pd.merge(data_next,data_goumailv_next_1,on='业务分类',how='left')

# 客单价
data_kedanjia_next_1=data_kedanjia_next.copy()
data_kedanjia_next_1['客单价']=data_kedanjia_next_1['客单价'].astype('int')

data_next=pd.merge(data_next,data_kedanjia_next_1,on='业务分类',how='left')

# GMV转化
data_gmv_next=data_baoguang_next_1.copy()
data_gmv_next['GMV转化']=data_leads_next_1['购买人次/leads']*data_zhuanhualv_next_1['转化率']*data_kedanjia_next_1['客单价']
data_gmv_next.drop(columns='首页资源位曝光UV',inplace=True)

data_next=pd.merge(data_next,data_gmv_next,on='业务分类',how='left')

# GMV直售
data_gmv1_next=data_baoguang_next_1.copy()
data_gmv1_next['GMV直售']=0
data_gmv1_next.drop(columns='首页资源位曝光UV',inplace=True)

data_next=pd.merge(data_next,data_gmv1_next,on='业务分类',how='left')

# GMV总计
data_next['GMV总计']=data_next['GMV转化']+data_next['GMV直售']

# 变现效率
data_next['变现效率']=data_next['GMV总计']/data_next['首页资源位曝光UV']

# 汇总
data_next_all=data_next.copy()
data_next_all['大类']='全真'
data_next_all['日期']=start_date+'-'+end_date
data_next_all=data_next_all[['日期','大类','业务分类', '累计日活', '首页资源位曝光UV', '购买人次/leads', '转化率', '曝光渗透率', '曝光购买率','客单价', 'GMV转化', 'GMV直售','GMV总计','变现效率']]

data_next_all


In [ ]:
# 图书
# 首页日志
data_rihuo_tushu=data_rihuo.copy()
data_rihuo_tushu['业务分类']='图书'
data_rihuo_tushu=data_rihuo_tushu[['业务分类','累计日活']]
data_rihuo_tushu['累计日活']=data_rihuo_tushu['累计日活'].astype('int')

# 曝光UV
data_baoguang_tushu_1=data_baoguang_tushu.groupby('业务分类')['曝光UV'].agg(['sum']).reset_index()
data_baoguang_tushu_1.columns=['业务分类','首页资源位曝光UV']
data_baoguang_tushu_1['首页资源位曝光UV']=data_baoguang_tushu_1['首页资源位曝光UV'].astype('int')

data_tushu=pd.merge(data_rihuo_tushu,data_baoguang_tushu_1,on='业务分类',how='left')

# 购买人次/leads
# data_leads_tushu_1=data_baoguang_tushu_1.copy()
# data_leads_tushu_1['购买人次/leads']=0
# data_leads_tushu_1.drop(columns='首页资源位曝光UV',inplace=True)
# data_leads_tushu_1['购买人次/leads']=data_leads_tushu_1['购买人次/leads'].astype('int')

data_leads_tushu_1=data_leads_tushu.copy()
data_leads_tushu_1.columns=['业务分类','购买人次/leads']
data_leads_tushu_1['购买人次/leads']=data_leads_tushu_1['购买人次/leads'].astype('int')

data_tushu=pd.merge(data_tushu,data_leads_tushu_1,on='业务分类',how='left')

# 转化率
data_zhuanhualv_tushu_1=data_baoguang_tushu_1.copy()
data_zhuanhualv_tushu_1['转化率']=1
data_zhuanhualv_tushu_1.drop(columns='首页资源位曝光UV',inplace=True)
data_zhuanhualv_tushu_1['转化率']=data_zhuanhualv_tushu_1['转化率'].astype('float')

data_tushu=pd.merge(data_tushu,data_zhuanhualv_tushu_1,on='业务分类',how='left')

# 曝光渗透率
data_shentoulv_tushu_1=data_baoguang_tushu_1.copy()
data_shentoulv_tushu_1['曝光渗透率']=data_baoguang_tushu_1['首页资源位曝光UV']/data_rihuo_tushu['累计日活']
data_shentoulv_tushu_1.drop(columns='首页资源位曝光UV',inplace=True)
data_shentoulv_tushu_1['曝光渗透率']=data_shentoulv_tushu_1['曝光渗透率'].astype('float')

data_tushu=pd.merge(data_tushu,data_shentoulv_tushu_1,on='业务分类',how='left')

# 曝光购买率
data_goumailv_tushu_1=data_leads_tushu_1.copy()
data_goumailv_tushu_1['曝光购买率']=data_leads_tushu_1['购买人次/leads']/data_baoguang_tushu_1['首页资源位曝光UV']
data_goumailv_tushu_1.drop(columns='购买人次/leads',inplace=True)
data_goumailv_tushu_1['曝光购买率']=data_goumailv_tushu_1['曝光购买率'].astype('float')

data_tushu=pd.merge(data_tushu,data_goumailv_tushu_1,on='业务分类',how='left')

# 客单价
data_kedanjia_tushu_1=data_baoguang_tushu_1.copy()
data_kedanjia_tushu_1['客单价']=0
data_kedanjia_tushu_1.drop(columns='首页资源位曝光UV',inplace=True)
data_kedanjia_tushu_1['客单价']=data_kedanjia_tushu_1['客单价'].astype('int')

data_tushu=pd.merge(data_tushu,data_kedanjia_tushu_1,on='业务分类',how='left')

# GMV转化
data_gmv_tushu=data_baoguang_tushu_1.copy()
data_gmv_tushu['GMV转化']=0
data_gmv_tushu.drop(columns='首页资源位曝光UV',inplace=True)

data_tushu=pd.merge(data_tushu,data_gmv_tushu,on='业务分类',how='left')

# GMV直售
# data_gmv1_tushu

data_tushu=pd.merge(data_tushu,data_gmv1_tushu,on='业务分类',how='left')

# GMV总计
data_tushu['GMV总计']=data_tushu['GMV转化']+data_tushu['GMV直售']

# 变现效率
data_tushu['变现效率']=data_tushu['GMV总计']/data_tushu['首页资源位曝光UV']

# 汇总
data_tushu_all=data_tushu.copy()
data_tushu_all['大类']='图书'
data_tushu_all['日期']=start_date+'-'+end_date
data_tushu_all=data_tushu_all[['日期','大类','业务分类', '累计日活', '首页资源位曝光UV', '购买人次/leads', '转化率', '曝光渗透率', '曝光购买率','客单价', 'GMV转化', 'GMV直售','GMV总计','变现效率']]
data_tushu_all['客单价']=data_tushu_all['GMV总计']/data_tushu_all['购买人次/leads']
data_tushu_all['客单价']=data_tushu_all['客单价'].astype('float').astype('int')
data_tushu_all


In [ ]:
# 创新
# 首页日志
data_rihuo_chuangxin=data_rihuo.copy()
data_rihuo_chuangxin['大类']='创新'
data_rihuo_chuangxin['累计日活']=data_rihuo_chuangxin['累计日活'].astype('int')

# 曝光UV
data_baoguang_chuangxin_1=data_baoguang_chuangxin.groupby('业务分类')['曝光UV'].agg(['sum']).reset_index()
data_baoguang_chuangxin_1.columns=['业务分类','首页资源位曝光UV']
data_baoguang_chuangxin_1['大类']='创新'
data_baoguang_chuangxin_1['首页资源位曝光UV']=data_baoguang_chuangxin_1['首页资源位曝光UV'].astype('int')

data_chuangxin=pd.merge(data_baoguang_chuangxin_1,data_rihuo_chuangxin,on='大类',how='left')

# 购买人次/leads
data_leads_chuangxin_1=data_leads_chuangxin.copy()
data_leads_chuangxin_1.columns=['业务分类','购买人次/leads']
data_leads_chuangxin_1['购买人次/leads']=data_leads_chuangxin_1['购买人次/leads'].astype('int')

data_chuangxin=pd.merge(data_chuangxin,data_leads_chuangxin_1,on='业务分类',how='left')

# 转化率
data_zhuanhualv_chuangxin_1=data_zhuanhualv_chuagnxin.copy()
data_zhuanhualv_chuangxin_1['转化率']=data_zhuanhualv_chuangxin_1['转化率'].astype('float')

data_chuangxin=pd.merge(data_chuangxin,data_zhuanhualv_chuangxin_1,on='业务分类',how='left')

# 曝光渗透率
data_chuangxin['曝光渗透率']=data_chuangxin['首页资源位曝光UV']/data_chuangxin['累计日活']
data_chuangxin['曝光渗透率']=data_chuangxin['曝光渗透率'].astype('float')

# 曝光购买率
data_chuangxin['曝光购买率']=data_chuangxin['购买人次/leads']/data_chuangxin['首页资源位曝光UV']
data_chuangxin['曝光购买率']=data_chuangxin['曝光购买率'].astype('float')

# 客单价
data_kedanjia_chuangxin_1=data_kedanjia_chuangxin.copy()
data_kedanjia_chuangxin_1['客单价']=data_kedanjia_chuangxin_1['客单价'].astype('int')

data_chuangxin=pd.merge(data_chuangxin,data_kedanjia_chuangxin_1,on='业务分类',how='left')
data_chuangxin['客单价']=data_chuangxin['客单价'].fillna(0)

# GMV转化
data_chuangxin['GMV转化']=data_chuangxin['购买人次/leads']*data_chuangxin['转化率']*data_chuangxin['客单价']

# GMV直售
# data_gmv1_chuangxin

data_chuangxin=pd.merge(data_chuangxin,data_gmv1_chuangxin[['业务分类','GMV直售']],on='业务分类',how='left')

# GMV总计
data_chuangxin['GMV直售']=data_chuangxin['GMV直售'].fillna(0)
data_chuangxin['GMV总计']=data_chuangxin['GMV转化']+data_chuangxin['GMV直售']

# 变现效率
data_chuangxin['变现效率']=data_chuangxin['GMV总计']/data_chuangxin['首页资源位曝光UV']
data_chuangxin['变现效率']=data_chuangxin['变现效率'].astype('float')

# 汇总
data_chuangxin_all=data_chuangxin.copy()
data_chuangxin_all['日期']=start_date+'-'+end_date
data_chuangxin_all=data_chuangxin_all[['日期','大类','业务分类', '累计日活', '首页资源位曝光UV', '购买人次/leads', '转化率', '曝光渗透率', '曝光购买率','客单价', 'GMV转化','GMV直售','GMV总计', '变现效率']]

data_chuangxin_all


In [ ]:
# 素质导流课
# 首页日志
np.set_printoptions(suppress=True)
data_rihuo_suzhi=data_rihuo.copy()
data_rihuo_suzhi['大类']='素质导流课'
data_rihuo_suzhi['累计日活']=data_rihuo_suzhi['累计日活'].astype('int')

# 曝光UV
data_baoguang_suzhi_1=data_baoguang_suzhi.groupby('业务分类')['曝光UV'].agg(['sum']).reset_index()
data_baoguang_suzhi_1.columns=['业务分类','首页资源位曝光UV']
data_baoguang_suzhi_1['大类']='素质导流课'
data_baoguang_suzhi_1['首页资源位曝光UV']=data_baoguang_suzhi_1['首页资源位曝光UV'].astype('int')

data_suzhi=pd.merge(data_baoguang_suzhi_1,data_rihuo_suzhi,on='大类',how='left')

# 购买人次/leads
data_leads_suzhi_1=data_leads_suzhi.copy()
data_leads_suzhi_1.columns=['业务分类','购买人次/leads']
data_leads_suzhi_1['购买人次/leads']=data_leads_suzhi_1['购买人次/leads'].astype('int')

data_suzhi=pd.merge(data_suzhi,data_leads_suzhi_1,on='业务分类',how='left')

# 转化率
data_zhuanhualv_suzhi_1=data_zhuanhualv_suzhi.copy()
data_zhuanhualv_suzhi_1['转化率']=data_zhuanhualv_suzhi_1['转化率'].astype('float')

data_suzhi=pd.merge(data_suzhi,data_zhuanhualv_suzhi_1,on='业务分类',how='left')

# 曝光渗透率
data_suzhi['曝光渗透率']=data_suzhi['首页资源位曝光UV']/data_suzhi['累计日活']
data_suzhi['曝光渗透率']=data_suzhi['曝光渗透率'].astype('float')

# 曝光购买率
data_suzhi['曝光购买率']=data_suzhi['购买人次/leads']/data_suzhi['首页资源位曝光UV']
data_suzhi['曝光购买率']=data_suzhi['曝光购买率'].astype('float')

# 客单价
data_kedanjia_suzhi_1=data_kedanjia_suzhi.copy()
data_kedanjia_suzhi_1['客单价']=data_kedanjia_suzhi_1['客单价'].astype('int')

data_suzhi=pd.merge(data_suzhi,data_kedanjia_suzhi_1,on='业务分类',how='left')

# GMV转化
data_suzhi['GMV转化']=data_suzhi['购买人次/leads']*data_suzhi['转化率']*data_suzhi['客单价']

# GMV直售
data_gmv1_suzhi=data_baoguang_suzhi_1.copy()
data_gmv1_suzhi['GMV直售']=0
data_gmv1_suzhi.drop(columns='首页资源位曝光UV',inplace=True)
data_gmv1_suzhi.drop(columns='大类',inplace=True)

data_suzhi=pd.merge(data_suzhi,data_gmv1_suzhi,on='业务分类',how='left')

# GMV总结
data_suzhi['GMV总计']=data_suzhi['GMV转化']+data_suzhi['GMV直售']

# 变现效率
data_suzhi['变现效率']=data_suzhi['GMV总计']/data_suzhi['首页资源位曝光UV']
data_suzhi['变现效率']=data_suzhi['变现效率'].astype('double')

# 汇总
data_suzhi_all=data_suzhi.copy()
data_suzhi_all['日期']=start_date+'-'+end_date
data_suzhi_all=data_suzhi_all[['日期','大类','业务分类', '累计日活', '首页资源位曝光UV', '购买人次/leads', '转化率', '曝光渗透率', '曝光购买率','客单价', 'GMV转化','GMV直售','GMV总计', '变现效率']]

data_suzhi_all



In [ ]:
# 老带新
# 首页日志
data_rihuo_laodaixin=data_rihuo.copy()
data_rihuo_laodaixin['大类']='老带新'
data_rihuo_laodaixin['累计日活']=data_rihuo_laodaixin['累计日活'].astype('int')

# 曝光UV
data_baoguang_laodaixin_1=data_baoguang_laodaixin.groupby('业务分类')['曝光UV'].agg(['sum']).reset_index()
data_baoguang_laodaixin_1.columns=['业务分类','首页资源位曝光UV']
data_baoguang_laodaixin_1['大类']='老带新'
data_baoguang_laodaixin_1['首页资源位曝光UV']=data_baoguang_laodaixin_1['首页资源位曝光UV'].astype('int')

data_laodaixin=pd.merge(data_baoguang_laodaixin_1,data_rihuo_laodaixin,on='大类',how='left')

# 购买人次/leads
data_leads_laodaixin_1=data_leads_laodaixin.copy()
data_leads_laodaixin_1.columns=['业务分类','购买人次/leads']
data_leads_laodaixin_1['购买人次/leads']=data_leads_laodaixin_1['购买人次/leads'].astype('int')

data_laodaixin=pd.merge(data_laodaixin,data_leads_laodaixin_1,on='业务分类',how='left')

# 转化率
data_zhuanhualv_laodaixin_1=data_zhuanhualv_laodaixin.copy()
data_zhuanhualv_laodaixin_1['转化率']=data_zhuanhualv_laodaixin_1['转化率'].astype('float')

data_laodaixin=pd.merge(data_laodaixin,data_zhuanhualv_laodaixin_1,on='业务分类',how='left')

# 曝光渗透率
data_laodaixin['曝光渗透率']=data_laodaixin['首页资源位曝光UV']/data_laodaixin['累计日活']
data_laodaixin['曝光渗透率']=data_laodaixin['曝光渗透率'].astype('float')

# 曝光购买率
data_laodaixin['曝光购买率']=data_laodaixin['购买人次/leads']/data_laodaixin['首页资源位曝光UV']
data_laodaixin['曝光购买率']=data_laodaixin['曝光购买率'].astype('float')

# 客单价
data_kedanjia_laodaixin_1=data_kedanjia_laodaixin.copy()
data_kedanjia_laodaixin_1['客单价']=data_kedanjia_laodaixin_1['客单价'].astype('int')

data_laodaixin=pd.merge(data_laodaixin,data_kedanjia_laodaixin_1,on='大类',how='left')

# GMV转化
data_laodaixin['GMV转化']=data_laodaixin['购买人次/leads']*data_laodaixin['转化率']*data_laodaixin['客单价']

# GMV直售
data_gmv1_laodaixin=data_baoguang_laodaixin_1.copy()
data_gmv1_laodaixin['GMV直售']=0
data_gmv1_laodaixin.drop(columns='首页资源位曝光UV',inplace=True)
data_gmv1_laodaixin.drop(columns='大类',inplace=True)

data_laodaixin=pd.merge(data_laodaixin,data_gmv1_laodaixin,on='业务分类',how='left')

# GMV总结
data_laodaixin['GMV总计']=data_laodaixin['GMV转化']+data_laodaixin['GMV直售']

# 变现效率
data_laodaixin['变现效率']=data_laodaixin['GMV总计']/data_laodaixin['首页资源位曝光UV']
data_laodaixin['变现效率']=data_laodaixin['变现效率'].astype('double')

# 汇总
data_laodaixin_all=data_laodaixin.copy()
data_laodaixin_all['日期']=start_date+'-'+end_date
data_laodaixin_all=data_laodaixin_all[['日期','大类','业务分类', '累计日活', '首页资源位曝光UV', '购买人次/leads', '转化率', '曝光渗透率', '曝光购买率','客单价', 'GMV转化','GMV直售','GMV总计', '变现效率']]

data_laodaixin_all


In [ ]:
data_all=pd.concat([data_suzhi_all,data_chuangxin_all,data_tushu_all,data_next_all,data_box_all,data_laodaixin_all])
data_all['GMV总计']=data_all['GMV总计'].astype('float').astype('int')
data_all['正价人次']=data_all['购买人次/leads']*data_all['转化率']
data_all['正价人次']=data_all['正价人次'].astype('float').astype('int')
data_all=data_all[['日期', '大类', '业务分类', '累计日活','曝光渗透率', '首页资源位曝光UV','曝光购买率', '购买人次/leads', '转化率'
                   ,'正价人次', '客单价', 'GMV转化','GMV直售','GMV总计', '变现效率']]
data_all.columns=['日期', '大类', '业务分类', '累计日活','曝光渗透率', '首页资源位曝光UV','报名/曝光率', '报名人次(leads)'
                  , 'leads转正(转化率)','正价人次', '客单价',  'GMV转化','GMV直售','GMV总计', '变现效率']
data_all['1万曝光产生的GMV']=data_all['变现效率']*10000
data_all['1万曝光产生的GMV']=data_all['1万曝光产生的GMV'].astype('float').astype('int')
data_all=data_all.sort_values(by='变现效率',ascending=False)
data_all.to_excel('%s-%s变现效率.xlsx' % (start_date,end_date))
data_all
